In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.utils import *
from delta import *
import hashlib
import shutil
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  

In [8]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("NB_DIM_Wallet") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Spark will automatically use the master specified in spark-defaults.conf
spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark

In [9]:
dfconfig=pd.read_csv('/home/jovyan/Notebooks/configpath.csv')
dfpath=spark.createDataFrame(dfconfig)

In [10]:
trgt_path_processed = dfpath.filter(col("DataFeedName") == "Fact_Delta_Path").select('Path').collect()[0][0]
trgt_path_csv = dfpath.filter(col("DataFeedName") == "Fact_CSV_Path").select('Path').collect()[0][0]
source_path = dfpath.filter(col("DataFeedName") == "FactFile").select('Path').collect()[0][0]
sheet_name =dfpath.filter(col("DataFeedName") == "sheet_name").select('Path').collect()[0][0]


In [11]:
# Read the Excel file (use Spark-Excel library)
df = pd.read_excel(source_path, sheet_name = sheet_name)
df = spark.createDataFrame(df)
df= df.withColumn("Date", to_date(df["Date"],"yyyy-MM-dd"))

In [12]:
df.createOrReplaceTempView("vw_src")
query = "select distinct `Wallet used` from vw_src"
print(query)
df_src = spark.sql(query)
df_src.show()

select distinct `Wallet used` from vw_src
+--------------------+
|         Wallet used|
+--------------------+
|               G-pay|
|       Amazon Wallet|
|           ICICI Pay|
|Credit Card - Amazon|
+--------------------+



In [13]:
df_output = \
df_src.withColumn("Walletsk",xxhash64("Wallet used")).withColumnRenamed("Wallet used","WalletUsed")

In [14]:
df_output.createOrReplaceTempView("vw_source")
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    column_name = df_output.columns
    set_clause = ", ".join([f"target.{i} = source.{i}" for i in column_name])
    insert_clause=",".join(column_name)
    insert_values=",".join([f"source.{i}" for i in column_name])
    query = f"""MERGE INTO delta.`{trgt_path_processed}` AS target 
            USING vw_source AS source 
            ON target.Walletsk = source.Walletsk 
            AND target.RowSK <> source.RowSK 
            WHEN MATCHED THEN UPDATE SET {set_clause}
            WHEN NOT MATCHED THEN INSERT ({insert_clause}) VALUES ({insert_values})"""
else :
    query=f"CREATE TABLE delta.`{trgt_path_processed}` USING DELTA AS SELECT * FROM vw_source"
print(query)
spark.sql(query)

MERGE INTO delta.`/mnt/Fact/Fact_Parquet/` AS target USING vw_source AS source ON target.categorysk = source.categorysk AND target.RowSK <> source.RowSK WHEN MATCHED THEN UPDATE SET target.WalletUsed = source.WalletUsed, target.Walletsk = source.Walletsk


AnalysisException: [DELTA_MERGE_UNRESOLVED_EXPRESSION] Cannot resolve source.categorysk in search condition given columns target.SpendingItem, target.SpendAmount, target.WalletSK, target.categorysk, target.DateSK, target.PKSK, target.UpdateTimeStamp, target.RowSK, source.WalletUsed, source.Walletsk; line 1 pos 102

In [11]:
# Save the DataFrame to a CSV file
spark.read.format("delta").load(trgt_path_processed).coalesce(1).write.format("csv").option("header","true").mode("overwrite").save(trgt_path_csv)

In [ ]:
trgt_copy_path = trgt_path_csv + "processed.csv"
files=os.listdir(trgt_path_csv)
selected_files = [file for file in files if file.startswith('part-00') and file.endswith('.csv')]
file=trgt_path_csv + selected_files[0]
print(selected_files)
shutil.copy(file, trgt_copy_path)

In [ ]:
delete_log = [file for file in files if "processed.csv" != file ]
for file in delete_log :
    os.remove(trgt_path_csv + file)
    print(f"removed {trgt_path_csv + file}")